# 📊 Medical Claims Data Ingestion Pipeline

**Project:** Predicting Paid Amount for Medical Claims  
**Stage:** Data Ingestion & ETL Pipeline  

---

## Overview

This notebook implements a production-grade data ingestion pipeline for processing ~17 million medical claims records:

1. **Chunked Data Loading** - Memory-efficient processing of 3.7GB+ files
2. **Claim ID Sampling** - Stratified sampling of 1M unique claims
3. **Data Validation** - Quality checks and data profiling
4. **AWS S3 Integration** - Cloud storage for processed data
5. **Reference Table Loading** - Loading lookup tables for enrichment

In [ ]:
# Standard library imports
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Third-party imports
import pandas as pd
import numpy as np

# Project imports
from src.config import ConfigurationManager, get_config
from src.utils.logger import setup_logging, get_logger, PipelineLogger
from src.utils.helpers import timer, memory_usage, format_bytes
from src.data.data_loader import DataLoader, ChunkedDataLoader, ReferenceDataLoader
from src.data.data_validator import DataValidator, validate_claims_data

# Setup logging
setup_logging(log_level="INFO", log_file="logs/data_ingestion.log")
logger = get_logger(__name__)

print("✓ Imports successful")
print(f"Current Memory Usage: {memory_usage()['rss']}")

In [ ]:
# Load configuration
try:
    config_manager = ConfigurationManager(str(project_root / "config" / "settings.yaml"))
    config = config_manager.config
    print(f"✓ Loaded configuration for: {config.name}")
    print(f"  Version: {config.version}")
except FileNotFoundError:
    print("⚠ Config file not found, using defaults")
    RAW_DATA_FILE = "PUBLICUSE_CLAIM_MC_2016.txt"
    CHUNK_SIZE = 100000
    SAMPLE_SIZE = 1000000
    DELIMITER = "|"
    RANDOM_SEED = 42

# Define paths
DATA_DIR = project_root / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
INTERIM_DIR = DATA_DIR / "interim"
REF_TABLES_DIR = project_root / "PUBLICUSE_REF_TABLES"

# Create directories
for dir_path in [RAW_DIR, PROCESSED_DIR, INTERIM_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print("\n📁 Directory Structure:")
print(f"  Raw Data:      {RAW_DIR}")
print(f"  Processed:     {PROCESSED_DIR}")
print(f"  Interim:       {INTERIM_DIR}")

## 2. Load Reference Tables

In [ ]:
# Initialize reference data loader
ref_loader = ReferenceDataLoader(str(REF_TABLES_DIR))

# Load all reference tables
with PipelineLogger("Loading Reference Tables", logger):
    reference_tables = ref_loader.load_all_reference_tables()

print(f"\n📋 Loaded {len(reference_tables)} reference tables:")
for name, df in reference_tables.items():
    print(f"  • {name}: {len(df):,} records, {len(df.columns)} columns")

## 3. Chunked Data Loading

The raw claims file is ~3.7GB with ~17 million rows. We use chunked loading for memory efficiency.

In [ ]:
# Configuration for data loading
RAW_DATA_PATH = RAW_DIR / "PUBLICUSE_CLAIM_MC_2016.txt"
CHUNK_SIZE = 100000
TOTAL_ROWS = 16982295
SAMPLE_SIZE = 1000000
CLAIM_ID_COLUMN = "CLAIM_ID_KEY"
RANDOM_SEED = 42

# Initialize chunked loader
chunked_loader = ChunkedDataLoader(chunk_size=CHUNK_SIZE, delimiter="|")

print(f"📊 Data Loading Configuration:")
print(f"  Source File: {RAW_DATA_PATH}")
print(f"  Chunk Size: {CHUNK_SIZE:,}")
print(f"  Expected Rows: {TOTAL_ROWS:,}")
print(f"  Sample Size: {SAMPLE_SIZE:,}")

In [ ]:
# Sample claims data (or create demo data if file not found)
if RAW_DATA_PATH.exists():
    with PipelineLogger("Sampling Claims Data", logger):
        sampled_df = chunked_loader.sample_by_claim_ids(
            str(RAW_DATA_PATH),
            claim_id_column=CLAIM_ID_COLUMN,
            sample_size=SAMPLE_SIZE,
            random_seed=RANDOM_SEED
        )
    print(f"\n✓ Sampled Data: {len(sampled_df):,} rows, {sampled_df[CLAIM_ID_COLUMN].nunique():,} unique claims")
else:
    print("⚠ Raw data file not found. Creating demo data for demonstration...")
    np.random.seed(RANDOM_SEED)
    n_samples = 50000
    sampled_df = pd.DataFrame({
        'CLAIM_ID_KEY': np.random.randint(1, 20000, n_samples),
        'AGE': np.random.choice(['25', '35', '45', '55', '65', '75', '90+'], n_samples),
        'SEX': np.random.choice(['M', 'F'], n_samples),
        'AMT_BILLED': np.abs(np.random.exponential(1000, n_samples)),
        'AMT_PAID': np.abs(np.random.exponential(500, n_samples)),
        'AMT_DEDUCT': np.abs(np.random.exponential(100, n_samples)),
        'AMT_COINS': np.abs(np.random.exponential(50, n_samples)),
        'FORM_TYPE': np.random.choice(['P', 'I', 'O'], n_samples),
        'SV_STAT': np.random.choice(['P', 'D', 'R'], n_samples),
        'PRODUCT_TYPE': np.random.choice(['HMO', 'PPO', 'POS'], n_samples),
        'ICD_DIAG_01_PRIMARY': np.random.choice(['Z00', 'J06', 'M54', 'I10', 'K21', 'E11', 'F32'], n_samples),
        'CLIENT_LOS': np.random.choice([0, 1, 2, 3, np.nan], n_samples, p=[0.7, 0.1, 0.08, 0.07, 0.05]),
    })
    print(f"✓ Created demo data: {len(sampled_df):,} rows")

# Display preview
print("\n📋 Data Preview:")
sampled_df.head(10)

## 4. Data Validation

In [ ]:
# Run validation
with PipelineLogger("Data Validation", logger):
    validation_report = validate_claims_data(sampled_df)

print(f"\n{validation_report.summary()}")

# Missing values analysis
missing_stats = sampled_df.isnull().sum()
missing_pct = (missing_stats / len(sampled_df) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing_stats, 'Missing %': missing_pct})
missing_df = missing_df.sort_values('Missing Count', ascending=False)

print("\n📊 Missing Values Summary:")
print(f"  Columns with missing: {(missing_stats > 0).sum()}")
missing_df[missing_df['Missing Count'] > 0]

## 5. Save Processed Data

In [ ]:
# Save sampled data
loader = DataLoader()

# Save as Parquet (efficient columnar format)
parquet_path = INTERIM_DIR / "sampled_claims.parquet"
loader.save_parquet(sampled_df, str(parquet_path))

# Also save as CSV
csv_path = INTERIM_DIR / "sampled_claims.csv"
loader.save_csv(sampled_df, str(csv_path))

print(f"\n💾 Data Saved:")
print(f"  Parquet: {parquet_path}")
print(f"  CSV: {csv_path}")

# Summary
print("\n" + "="*60)
print("📊 DATA INGESTION SUMMARY")
print("="*60)
print(f"  Total Rows: {len(sampled_df):,}")
print(f"  Total Columns: {len(sampled_df.columns)}")
print(f"  Memory Usage: {sampled_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
if 'AMT_PAID' in sampled_df.columns:
    print(f"  Paid Amount - Mean: ${sampled_df['AMT_PAID'].mean():,.2f}")
print(f"\n✅ Data ingestion completed! Next: Run 02_exploratory_data_analysis.ipynb")